In [0]:
import pyspark
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType, FloatType

# A list of folders containing sample datasets we can use
display(dbutils.fs.ls("/databricks-datasets/samples/"))

path,name,size,modificationTime
dbfs:/databricks-datasets/samples/adam/,adam/,0,0
dbfs:/databricks-datasets/samples/data/,data/,0,0
dbfs:/databricks-datasets/samples/docs/,docs/,0,0
dbfs:/databricks-datasets/samples/lending_club/,lending_club/,0,0
dbfs:/databricks-datasets/samples/newsgroups/,newsgroups/,0,0
dbfs:/databricks-datasets/samples/people/,people/,0,0
dbfs:/databricks-datasets/samples/population-vs-price/,population-vs-price/,0,0


In [0]:
# Loading in a sample table into the dataframe
df = spark.read.csv("/databricks-datasets/samples/population-vs-price/data_geo.csv", header=True)

In [0]:
# To view the first 20 rows of the df
df.show()
# OR we can add an integer into the parentheses to view a specific 
# number of rows
df.show(2)

+---------+-------------+----------+----------+------------------------+-----------------------+
|2014 rank|         City|     State|State Code|2014 Population estimate|2015 median sales price|
+---------+-------------+----------+----------+------------------------+-----------------------+
|      101|   Birmingham|   Alabama|        AL|                  212247|                  162.9|
|      125|   Huntsville|   Alabama|        AL|                  188226|                  157.7|
|      122|       Mobile|   Alabama|        AL|                  194675|                  122.5|
|      114|   Montgomery|   Alabama|        AL|                  200481|                    129|
|       64|Anchorage[19]|    Alaska|        AK|                  301010|                   null|
|       78|     Chandler|   Arizona|        AZ|                  254276|                   null|
|       86|  Gilbert[20]|   Arizona|        AZ|                  239277|                   null|
|       88|     Glendale|   Ar

In [0]:
# Viewing the column names
df.columns

Out[19]: ['2014 rank',
 'City',
 'State',
 'State Code',
 '2014 Population estimate',
 '2015 median sales price']

In [0]:
df.withColumnRenamed('2014 rank', '2014_rank')
df.head(2)

Out[22]: [Row(2014 rank='101', City='Birmingham', State='Alabama', State Code='AL', 2014 Population estimate='212247', 2015 median sales price='162.9'),
 Row(2014 rank='125', City='Huntsville', State='Alabama', State Code='AL', 2014 Population estimate='188226', 2015 median sales price='157.7')]

In [0]:
df2 = df.withColumnRenamed('2014 rank', '2014_rank') \
        .withColumnRenamed('State Code', 'state_code') \
        .withColumnRenamed('2014 Population estimate', '2014_pop_estimate') \
        .withColumnRenamed('2015 median sales price', '2015_median_sales_price')
df2.select(['2014_rank', '2014_pop_estimate']).show()

+---------+-----------------+
|2014_rank|2014_pop_estimate|
+---------+-----------------+
|      101|           212247|
|      125|           188226|
|      122|           194675|
|      114|           200481|
|       64|           301010|
|       78|           254276|
|       86|           239277|
|       88|           237517|
|       38|           464704|
|      148|           166934|
|        6|          1537058|
|       95|           230512|
|      215|           126275|
|      142|           172816|
|       33|           527972|
|      119|           197706|
|       56|           346997|
|      261|           108930|
|       52|           368759|
|      227|           118853|
+---------+-----------------+
only showing top 20 rows



In [0]:
df2.printSchema()

root
 |-- 2014_rank: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- state_code: string (nullable = true)
 |-- 2014_pop_estimate: string (nullable = true)
 |-- 2015_median_sales_price: string (nullable = true)



In [0]:
# change value

df2.withColumn("2014_rank", col("2014_rank").cast(IntegerType()))

Out[30]: DataFrame[2014_rank: int, City: string, State: string, state_code: string, 2014_pop_estimate: string, 2015_median_sales_price: string]

In [0]:
df3 = df2.withColumn("2014_rank", col("2014_rank").cast(IntegerType())) \
         .withColumn("2014_pop_estimate", col("2014_pop_estimate").cast(IntegerType())) \
         .withColumn("2015_median_sales_price", col("2015_median_sales_price").cast(FloatType()))
# Giving our df3 the table name 'pop_price'
df3.createOrReplaceTempView('pop_price')


In [0]:
# Viewing the top 10 cities based on the '2014_rank' column
top_10_results = spark.sql("""SELECT * FROM pop_price 
                              WHERE 2014_rank <= 10
                              SORT BY 2014_rank ASC""")
top_10_results.show()

+---------+---------------+------------+----------+-----------------+-----------------------+
|2014_rank|           City|       State|state_code|2014_pop_estimate|2015_median_sales_price|
+---------+---------------+------------+----------+-----------------+-----------------------+
|        1|    New York[6]|    New York|        NY|          8491079|                  388.6|
|        2|    Los Angeles|  California|        CA|          3928864|                  434.7|
|        3|        Chicago|    Illinois|        IL|          2722389|                  192.5|
|        4|     Houston[7]|       Texas|        TX|          2239558|                  200.3|
|        5|Philadelphia[8]|Pennsylvania|        PA|          1560297|                  204.9|
|        6|        Phoenix|     Arizona|        AZ|          1537058|                  206.1|
|        7|    San Antonio|       Texas|        TX|          1436697|                  184.7|
|        8|      San Diego|  California|        CA|         